In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
from keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier

/home/janardhan/anaconda3/envs/dl/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [6]:
filenames = np.load('filenames_list.npy')

In [7]:
resnet_preds = np.load('resnet_preds.npy')
mobilenet_preds = np.load('mobilenet_preds.npy')
cacnn_preds = np.load('cacnn_preds.npy')
densenet_preds = np.load('densenet_valid_preds.npy')

In [8]:
resnet_preds_classes = np.argmax(resnet_preds,axis=1)
mobilenet_preds_classes = np.argmax(mobilenet_preds,axis=1)
cacnn_preds_classes = np.argmax(cacnn_preds,axis=1)
densenet_preds_classes = np.argmax(densenet_preds,axis=1)

In [9]:
mean_preds = (resnet_preds+mobilenet_preds+cacnn_preds+densenet_preds)/4

In [12]:
np.bincount(np.argmax(mean_preds,axis=1)==target_classes)

array([ 18, 856])

In [13]:
feature_arr_classes = np.array([mobilenet_preds_classes,resnet_preds_classes,cacnn_preds_classes,densenet_preds_classes])

In [14]:
max_voting = [np.argmax(np.bincount(feature_arr_classes[:,i])) for i in range(874)]

In [15]:
np.bincount(max_voting==target_classes)

array([ 21, 853])

In [15]:
cnt=0
for i in range(len(resnet_preds_classes)):
    if(mobilenet_preds_classes_224[i]!=mobilenet_preds_classes[i]):
        cnt = cnt +1

In [16]:
cacnn_preds_classes.shape

(874,)

In [11]:
target_classes = np.load('target_classes.npy')

In [18]:
target_classes_one_hot = to_categorical(target_classes)

In [16]:
feature_arr = np.concatenate([mobilenet_preds,resnet_preds,cacnn_preds,densenet_preds],axis=1)

In [18]:
features_df = pd.DataFrame(feature_arr)

In [101]:
X_train, X_test, y_train, y_test = train_test_split(feature_arr, target_classes, test_size = 0.2, random_state = 0)

In [102]:
print(X_train.shape[0])
print(X_test.shape[0])

699
175


In [186]:
index_list = []

In [187]:
for j in range(10):
    temp_index = []
    for index,i in enumerate(target_classes):
        if(i==j):
            temp_index.append(index)
    index_list.append(temp_index)

In [1]:
#ensembling with seperate logistic regression model for each class

In [189]:
model_lr = LogisticRegression(verbose=0,class_weight='balanced',multi_class='ovr',solver='newton-cg')

In [190]:
final_res = []

In [191]:
for i in range(10):
    temp_y_train = np.zeros((len(target_classes)))
    temp_y_train[index_list[i]]=1
        
    model_lr.fit(feature_arr[:,[i,i+10,i+20,i+30]],temp_y_train)
    final_res.append(model_lr.predict_proba(test_feature_arr[:,[i,i+10,i+20,i+30]]))

In [192]:
temp_final_res = []

In [193]:
len(mobilenet_test_preds)

2640

In [194]:
for i in range(len(mobilenet_test_preds)):
    temp_final_res.append(np.argmax(np.array([final_res[j][i][1] for j in range(10)])))
        

In [196]:
np.save('test_pred_classes_unalt.npy',temp_final_res)

In [75]:
#test preds

In [2]:
mobilenet_test_preds = np.load('mobilenet_test_preds.npy')
resnet_test_preds = np.load('resnet_test_preds.npy')
cacnn_test_preds = np.load('cacnn_test_preds.npy')
densenet_test_preds = np.load('densenet_512_test_preds.npy')

In [167]:
test_feature_arr = np.concatenate([mobilenet_test_preds,resnet_test_preds,cacnn_test_preds,densenet_test_preds],axis=1)

In [3]:
test_preds_mean = (mobilenet_test_preds+resnet_test_preds+cacnn_test_preds+densenet_test_preds)/4

In [4]:
test_preds_mean.shape

(2640, 10)

In [125]:
#mean, max-voting

In [143]:
valid_preds_rf = grid_obj_rf.best_estimator_.predict_proba(features_df)
valid_preds_lr = grid_obj_lr.best_estimator_.predict_proba(features_df)
valid_preds_dt = grid_obj_dt.best_estimator_.predict_proba(features_df)
valid_preds_svm = grid_obj_svm.best_estimator_.predict_proba(features_df)
valid_preds_xgb = grid_obj_xgb.best_estimator_.predict_proba(features_df)

In [144]:
valid_preds_mean = (valid_preds_rf+valid_preds_lr+valid_preds_dt+valid_preds_svm+valid_preds_xgb)/5

In [145]:
np.bincount(np.argmax(valid_preds_mean,axis=1)==target_classes)

array([  3, 871])

In [149]:
valid_preds_voting = np.array([np.argmax(valid_preds_rf,axis=1),np.argmax(valid_preds_lr,axis=1),np.argmax(valid_preds_dt,axis=1),np.argmax(valid_preds_svm,axis=1),np.argmax(valid_preds_xgb,axis=1)])

In [150]:
valid_preds_max_voting = [np.argmax(np.bincount(valid_preds_voting[:,i])) for i in range(874)]

In [151]:
np.bincount(valid_preds_max_voting==target_classes)

array([  8, 866])

In [171]:
#test preds

In [235]:
mobilenet_test_preds = np.load('mobilenet_test_preds.npy')
resnet_test_preds = np.load('resnet_test_preds.npy')
cacnn_test_preds = np.load('cacnn_test_preds.npy')
mobilenet_test_preds_224 = np.load('mobilenet_test_preds_224.npy')
resnet_test_preds_224 = np.load('resnet_test_preds_224.npy')
cacnn_test_preds_224 = np.load('cacnn_test_preds_224.npy')

In [11]:
test_filenames = np.load('test_filenames.npy')

In [237]:
test_preds = np.concatenate([mobilenet_test_preds,resnet_test_preds,cacnn_test_preds,mobilenet_test_preds_224,resnet_test_preds_224,cacnn_test_preds_224],axis=1)

In [174]:
test_features_df = pd.DataFrame(test_preds)

In [276]:
test_preds_mean = []

In [277]:
for i in range(len(test_filenames)):
    test_preds_mean.append(np.mean(np.array([mobilenet_test_preds[i],resnet_test_preds[i],cacnn_test_preds[i],mobilenet_test_preds_224[i],resnet_test_preds_224[i],cacnn_test_preds_224[i]]),axis=0))

In [278]:
test_preds_mean[0]

array([ 0.00779759,  0.03605797,  0.89843506,  0.00272399,  0.00640286,
        0.01698795,  0.00332917,  0.0033218 ,  0.01365902,  0.01128468], dtype=float32)

In [203]:
test_preds_rf = grid_obj_rf.best_estimator_.predict_proba(test_features_df)
test_preds_lr = grid_obj_lr.best_estimator_.predict_proba(test_features_df)
test_preds_dt = grid_obj_dt.best_estimator_.predict_proba(test_features_df)
test_preds_svm = grid_obj_svm.best_estimator_.predict_proba(test_features_df)
test_preds_xgb = grid_obj_xgb.best_estimator_.predict_proba(test_features_df)

In [204]:
test_preds_mean = (test_preds_rf+test_preds_lr+test_preds_dt+test_preds_svm+test_preds_xgb)/5

In [205]:
test_preds_voting = np.array([np.argmax(test_preds_rf,axis=1),np.argmax(test_preds_lr,axis=1),np.argmax(test_preds_dt,axis=1),np.argmax(test_preds_svm,axis=1),np.argmax(test_preds_xgb,axis=1)])

In [206]:
test_preds_max_voting = [np.argmax(np.bincount(test_preds_voting[:,i])) for i in range(len(test_filenames))]

In [220]:
final_test_preds = model_rf1.predict(test_preds)

In [5]:
class_map = {'HTC-1-M7': 0,
 'LG-Nexus-5x': 1,
 'Motorola-Droid-Maxx': 2,
 'Motorola-Nexus-6': 3,
 'Motorola-X': 4,
 'Samsung-Galaxy-Note3': 5,
 'Samsung-Galaxy-S4': 6,
 'Sony-NEX-7': 7,
 'iPhone-4s': 8,
 'iPhone-6': 9}

In [6]:
class_map = {class_map[i]:i for i in class_map.keys()}

In [7]:
class_map

{0: 'HTC-1-M7',
 1: 'LG-Nexus-5x',
 2: 'Motorola-Droid-Maxx',
 3: 'Motorola-Nexus-6',
 4: 'Motorola-X',
 5: 'Samsung-Galaxy-Note3',
 6: 'Samsung-Galaxy-S4',
 7: 'Sony-NEX-7',
 8: 'iPhone-4s',
 9: 'iPhone-6'}

In [8]:
final_test_preds = np.argmax(test_preds_mean,axis=1)

In [211]:
len(test_preds_max_voting)

2640

In [9]:
sub_preds = [class_map[i] for i in final_test_preds]

In [286]:
for index,i in enumerate(test_filenames):
    if(i.split('_')[-1]=='manip.tif'):
        sub_preds[index]=''

In [209]:
submission = pd.DataFrame({'fname':test_filenames,'camera':sub_preds})
submission.to_csv('submission_ensemble_log_indv_classes.csv', encoding="utf8", index=False)

In [210]:
from IPython.display import FileLink
FileLink('submission_ensemble_log_indv_classes.csv')

/home/janardhan/Documents/submission_ensemble_log_indv_classes.csv

In [2]:
# Private LB score - 97.28